## <span style="color:red">The Python-14 task solution</span>

<span style="color:red">The conditions:</span>  
<span style="color:red">There are two databases (two sheets of an Excel file): a database with competitors'</span>  
<span style="color:red">prices(Data_Parsing) and an internal company database (Data_Company).</span>  
<span style="color:red">There are two id's in the parsing base that uniquely identify the product:</span>  
<span style="color:red">producer_id and producer_color.</span>  
<span style="color:red">There are two similar fields in the company database: item_id and color_id.</span>  
<span style="color:red">We know that the codes in the two databases differ in the presence of a set of service characters.</span>  
<span style="color:red">The parsing base contains the following symbols: _, -, ~, \\, /.</span>  

<span style="color:red">A task:</span>  
<span style="color:red">Read data from Excel into DataFrame.</span>  
<span style="color:red">Pull data from the company's database (item_id, color_id, current_price)</span>  
<span style="color:red">to the parsing base and form a price difference column in % (competitor's price to company's price).</span>  
<span style="color:red">Determine strong deviations from the average in the price difference within the brand-category</span>  
<span style="color:red">(remove random outliers that greatly distort the comparison). The criterion by taste.</span>  
<span style="color:red">Write  base parsing Excel files with marked outliers -</span>  
<span style="color:red">one unsorted with outliers at the bottom of the table and the second sorted by index.</span>  
<span style="color:red">Receive a brilliant reward for a brilliant solution!</span>  

In [8]:
import numpy as np
import pandas as pd
import plotly.express as px
from re import sub
from py14_task import outliers_z_score

<span style="color:green">Getting DataFrames and initial data reviewing.</span>

In [9]:
pars_data = pd.read_excel('data/Data_TSUM.xlsx', sheet_name='Data_Parsing')
comp_data = pd.read_excel('data/Data_TSUM.xlsx', sheet_name='Data_Company')
display(pars_data.info(), comp_data.info())
display(pars_data.sample(2), comp_data.sample(2))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   brand           75 non-null     object
 1   Category        75 non-null     object
 2   producer_id     75 non-null     object
 3   producer_color  75 non-null     object
 4   price           75 non-null     int64 
dtypes: int64(1), object(4)
memory usage: 3.1+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   brand          75 non-null     object
 1   Category       75 non-null     object
 2   item_id        75 non-null     object
 3   color_id       75 non-null     object
 4   current price  75 non-null     int64 
dtypes: int64(1), object(4)
memory usage: 3.1+ KB


None

None

,brand,Category,producer_id,producer_color,price
30,Brioni,Shoes,bbbbb1111_16,bla__ck,190
51,Stone Island,Shoes,sssss1111_12,black,178


,brand,Category,item_id,color_id,current price
41,Brioni,Bags,bbbbb111127,red,183
43,Brioni,Bags,bbbbb111129,red,197


<span style="color:green">Data transformations.</span>

In [10]:
pars_data[[  # initial line cleanup
    'producer_color', 'producer_id'
]] = pars_data[['producer_color',
                'producer_id']].applymap(lambda x: sub(r'[\W_]', '', x))

# get merged DataFrame:
pars_data.rename(columns={
    'producer_id': 'item_id',
    'producer_color': 'color_id',
    'price': 'concurrent price'
},
                 inplace=True)
res_data = pd.merge(comp_data, pars_data, how='left')
res_data['ratio'] = res_data['concurrent price'].div(
    res_data['current price']).mul(100).astype('float32')

# type definitions:
cat_list = ['brand', 'Category', 'color_id']
int_list = ['current price', 'concurrent price']
for col in res_data.columns:
    if col in cat_list:
        res_data[col] = res_data[col].astype('category')
    elif col in int_list:
        res_data[col] = res_data[col].astype('int16')

res_data['item_id'] = res_data['item_id'].apply(  # final line cleanup
    lambda x: sub(r'([a-z])\1+', r'\1', x))

profit = 100 - (
    res_data.memory_usage().sum() /  # to show the memory usage profit
    (pars_data.memory_usage().sum() + comp_data.memory_usage().sum()) *
    100).round(decimals=2)
print(f'The memory usage profit after data transformation: {profit}%\n')

# res_data.to_excel('data/res_data.xlsx', index=False)  # uncomment to proceed
display(res_data.info(), res_data.sample(3))

The memory usage profit after data transformation: 61.43%

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75 entries, 0 to 74
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   brand             75 non-null     category
 1   Category          75 non-null     category
 2   item_id           75 non-null     object  
 3   color_id          75 non-null     category
 4   current price     75 non-null     int16   
 5   concurrent price  75 non-null     int16   
 6   ratio             75 non-null     float32 
dtypes: category(3), float32(1), int16(2), object(1)
memory usage: 2.4+ KB


None

,brand,Category,item_id,color_id,current price,concurrent price,ratio
27,Brioni,Shoes,b111113,black,235,269,114.468086
32,Brioni,Shoes,b111118,white,235,165,70.212769
16,Valentino,Bags,a111127,red,203,293,144.334976


<span style="color:green">Two box plots construction to get visual identification of explicit outliers splited by significant categories.</span>

In [11]:
fig = px.box(res_data,
             x='ratio',
             color='brand',
             facet_row='Category',
             width=1200,
             height=450)
fig.show()
# fig.write_html("plotly/fig_01.html") # done (for github)
fig = px.box(res_data,
             x='ratio',
             color='brand',
             facet_row='color_id',
             width=1200,
             height=600)
fig.show()
# fig.write_html("plotly/fig_02.html") # done (for github)

<center> <img src = https://github.com/ssergeegress/sf_experience/blob/main/tasks/py14_task/plotly/fig_01.png?raw=true alt='fig_01.png' style='width:1200px'>
<center> <img src = https://github.com/ssergeegress/sf_experience/blob/main/tasks/py14_task/plotly/fig_02.png?raw=true alt='fig_02.png' style='width:1200px'>

- <span style="color:blue">Hmm... It seems two black and one white shoes by Brioni are obvious outliers here!</span>

<span style="color:green">Let's try to find the proper sigma coefficients for these outliers on the z-method histogram:</span>

In [12]:
outliers, cleaned = outliers_z_score(
    res_data[(res_data['brand'] == 'Brioni')
             & (res_data['Category'] == 'Shoes')],
    'ratio',
    left=1.4,
    right=1.4,
    verb_info=True)

Математическое ожидание: 105.30249786376953, Стандартное отклонение: 27.621070861816406
Нижняя граница: 66.63299865722657, Верхняя граница: 143.9719970703125
Асимметрия: 0.4473887085914612, Правосторонняя
Число выбросов по методу z-отклонения: 3
Результирующее число записей: 11


<center> <img src = https://github.com/ssergeegress/sf_experience/blob/main/tasks/py14_task/plotly/fig_03.png?raw=true alt='fig_03.png' style='width:800px'>

- <span style="color:blue">With coefficient values of 1.5 in both directions, our outliers are finaly cut off, guys!</span>

<span style="color:green">Applying these values to the entire dataset will produce a likely reliable number of outliers for it:</span>

In [13]:
outliers, cleaned = outliers_z_score(res_data,
                                     'ratio',
                                     left=1.5,
                                     right=1.3,
                                     verb_info=True)

Математическое ожидание: 102.65980529785156, Стандартное отклонение: 28.810075759887695
Нижняя граница: 59.44469165802002, Верхняя граница: 140.11290378570556
Асимметрия: 0.5169171094894409, Правосторонняя
Число выбросов по методу z-отклонения: 11
Результирующее число записей: 64


<center> <img src = https://github.com/ssergeegress/sf_experience/blob/main/tasks/py14_task/plotly/fig_04.png?raw=true alt='fig_04.png' style='width:800px'>

- <span style="color:blue">Since the mean line is shifted relative to the median to the right compared to the shift on the previous histogram,</span>
- <span style="color:blue">we will shift the range by 0.1 to the left and and extremely funny get a match with the main feature distribution</span>
- <span style="color:blue">for the entire dataset. Thus, the obtained 11 outliers are the most likely reliable here, aren't they? =)</span>

<span style="color:green">Anyway getting the final DataFrame and writing the Excel files with 11 outliers (right now we have a courious bug with arounded float16|32 in Pandas, however export to Excel will reveal all rounded values properly):</span>

In [14]:
cleaned.insert(len(cleaned.columns), 'is_out', 'No')
outliers.insert(len(outliers.columns), 'is_out', 'Yes')
res_data = pd.concat([cleaned, outliers])
res_data['ratio'] = res_data['ratio'].round(2)
res_data_sorted = res_data.sort_index()
display(res_data_sorted.info(), res_data_sorted.sample(3))
# res_data.to_excel('data/DataTSUM_out.xlsx', sheet_name='Data_Parsing', index=False)  # uncomment to proceed
# res_data.sort_index().to_excel('data/DataTSUM_out_sorted.xlsx', sheet_name='Data_Parsing', index=False)  # uncomment to proceed

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75 entries, 0 to 74
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   brand             75 non-null     category
 1   Category          75 non-null     category
 2   item_id           75 non-null     object  
 3   color_id          75 non-null     category
 4   current price     75 non-null     int16   
 5   concurrent price  75 non-null     int16   
 6   ratio             75 non-null     float32 
 7   is_out            75 non-null     object  
dtypes: category(3), float32(1), int16(2), object(2)
memory usage: 2.9+ KB


None

,brand,Category,item_id,color_id,current price,concurrent price,ratio,is_out
25,Brioni,Shoes,b111111,black,247,263,106.480003,No
36,Brioni,Shoes,b111122,white,270,260,96.300003,No
27,Brioni,Shoes,b111113,black,235,269,114.470001,No
